**Importing historical data using CCXT**

In [2]:
!pip install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.3 MB/s eta 0:00:00


In [3]:
import time
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def get_history(symbol, interval, start = None, limit = 1000):
    
    if start:
        start = exchange.parse8601(start)
    
    data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
    last_bar_actual = data[-1][0] # timestamp of last loaded bar
    
    # timestamp of current bar 
    last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
    # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
    while last_bar_target != last_bar_actual: 
        
        time.sleep(0.1)
        data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval,
                                      since = last_bar_actual, limit = limit)
        data += data_add[1:]
        last_bar_actual = data[-1][0]
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
    raw = pd.DataFrame(data)
    raw.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    raw.Date = pd.to_datetime(raw.Date, unit = "ms")
    raw.set_index("Date", inplace = True)

    return raw

In [10]:
exchange = ccxt.binance()
symbol = "BTCUSDT"
interval = "15m"
start = "2023-01-01 00:00:00"

In [11]:
raw = get_history(symbol, interval, start)

In [12]:
raw

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16520.00,16520.69,1172.53835
2023-01-01 00:15:00,16521.26,16545.70,16517.72,16544.19,1102.62888
2023-01-01 00:30:00,16544.19,16544.61,16508.39,16515.43,1365.65633
2023-01-01 00:45:00,16515.91,16536.84,16515.43,16529.67,724.01214
2023-01-01 01:00:00,16529.59,16541.80,16525.78,16538.21,977.24680
...,...,...,...,...,...
2023-05-02 20:00:00,28686.13,28727.75,28626.85,28661.87,548.53753
2023-05-02 20:15:00,28661.87,28728.97,28622.44,28705.09,562.76329
2023-05-02 20:30:00,28705.09,28788.99,28697.41,28763.19,394.89958


In [13]:
raw.rename(columns={"Open":"open", "High":"high" ,"Low" : "low", "Close":"close","Volume":"volume"}, inplace = True)
dataframe = raw.copy()
dataframe

,open,high,low,close,volume
Date,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16520.00,16520.69,1172.53835
2023-01-01 00:15:00,16521.26,16545.70,16517.72,16544.19,1102.62888
2023-01-01 00:30:00,16544.19,16544.61,16508.39,16515.43,1365.65633
2023-01-01 00:45:00,16515.91,16536.84,16515.43,16529.67,724.01214
2023-01-01 01:00:00,16529.59,16541.80,16525.78,16538.21,977.24680
...,...,...,...,...,...
2023-05-02 20:00:00,28686.13,28727.75,28626.85,28661.87,548.53753
2023-05-02 20:15:00,28661.87,28728.97,28622.44,28705.09,562.76329
2023-05-02 20:30:00,28705.09,28788.99,28697.41,28763.19,394.89958


# Pandas Data Type Mismatch

This notebook explains how to identify data in columns with the wrong data type with `pandas`.

### Packages

This tutorial uses:
* [pandas](https://pandas.pydata.org/docs/)
* [datetime](https://docs.python.org/3/library/datetime.html)

In [ ]:
import pandas as pd
import datetime
import numpy as np

## Creating the data

We will create a dataframe that contains multiple occurances of duplication for this example.

In [ ]:
df = pd.DataFrame({'A': ['text']*20,
                   'B': [1, 2.2]*10,
                   'C': [True, False]*10,
                   'D': pd.to_datetime('2020-01-01')
                  })

Next, add some mistyped data to the dataframe.

In [ ]:
df.iloc[0,0] = 1
df.iloc[1,0] = -2
df.iloc[10,0] = pd.to_datetime('2021-01-01')
df.iloc[5,1] = '2.2'
df.iloc[7,1] = 'A+B'
df.iloc[4,2] = 1
df.iloc[5,2] = 'False'
df.iloc[9,2] = -12.6
df.iloc[12,2] = 'text'
df.iloc[2,3] = 12
df.iloc[12,3] = '2020-01-01'
df

,A,B,C,D
0,1,1.0,True,2020-01-01 00:00:00
1,-2,2.2,False,2020-01-01 00:00:00
2,text,1.0,True,12
3,text,2.2,False,2020-01-01 00:00:00
4,text,1.0,1,2020-01-01 00:00:00
5,text,2.2,False,2020-01-01 00:00:00
6,text,1.0,True,2020-01-01 00:00:00
7,text,A+B,False,2020-01-01 00:00:00
8,text,1.0,True,2020-01-01 00:00:00
9,text,2.2,-12.6,2020-01-01 00:00:00


## Identify mistyped data

The function `applymap` and `isinstance` will return a Boolean dataframe with **True** when the data type matches and **False** when the data type does not match.

### Check numeric

In [ ]:
numeric = df.applymap(lambda x: isinstance(x, (int, float)))
numeric

,A,B,C,D
0,True,True,True,False
1,True,True,True,False
2,False,True,True,True
3,False,True,True,False
4,False,True,True,False
5,False,False,False,False
6,False,True,True,False
7,False,False,True,False
8,False,True,True,False
9,False,True,True,False


Since only column **B** is supposed to be numeric, this can be made more specific by running `applymap` only on column **B**.

In [ ]:
numeric = df.applymap(lambda x: isinstance(x, (int, float)))['B']
numeric

0      True
1      True
2      True
3      True
4      True
5     False
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: B, dtype: bool

Using this Boolean series to return the non-numeric data

In [ ]:
df[~numeric]

,A,B,C,D
5,text,2.2,False,2020-01-01 00:00:00
7,text,A+B,False,2020-01-01 00:00:00


### Check datetime

In [ ]:
dt = df.applymap(lambda x: isinstance(x, (datetime.datetime)))['D']
dt

0      True
1      True
2     False
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12    False
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: D, dtype: bool

Using this Boolean series to return the non-numeric data

In [ ]:
df[~dt]

,A,B,C,D
2,text,1.0,True,12
12,text,1.0,text,2020-01-01


### Check string

In [ ]:
strings = df.applymap(lambda x: isinstance(x, (str)))['A']
strings

0     False
1     False
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10    False
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: A, dtype: bool

Using this Boolean series to return the non-numeric data

In [ ]:
df[~strings]

,A,B,C,D
0,1,1.0,True,2020-01-01 00:00:00
1,-2,2.2,False,2020-01-01 00:00:00
10,2021-01-01 00:00:00,1.0,True,2020-01-01 00:00:00


### Check Boolean

In [ ]:
torf = df.applymap(lambda x: isinstance(x, (bool)))['C']
torf

0      True
1      True
2      True
3      True
4     False
5     False
6      True
7      True
8      True
9     False
10     True
11     True
12    False
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: C, dtype: bool

Using this Boolean series to return the non-numeric data

In [ ]:
df[~torf]

,A,B,C,D
4,text,1.0,1,2020-01-01 00:00:00
5,text,2.2,False,2020-01-01 00:00:00
9,text,2.2,-12.6,2020-01-01 00:00:00
12,text,1.0,text,2020-01-01
